## 앙상블 학습

- 앙상블 학습의 유형은 보팅, 배깅, 부스팅 세가지로 나눌 수 있으며 이외에도 스태깅을 포함한 다양한 앙상블 방법이 있다.

- 보팅의 경우 다른 알고리즘을 가진 분류기를 결합하는 것이고 배깅의 경우 각각의 분류기가 모두 같은 유형의 알고리즘 기반이다.

- 정형 데이터의 예측 분석 영역에서는 매우 높은 예측 성능, bagging과 Boosting

- 배깅 방식의 대표인 Random Forest는 뛰어난 예측 성능, 상대적으로 빠른 수행시간, 유연성 등으로 애용

- 부스팅의 효시는 Gradient Boosting, 한 단계 발전시키면서도 시간 단축시킨 XgBoost, LightGBM 이 정형 데이터의 분류 영역에서 활용도 확대

- 앙상블의 앙상블이라고 불리는 스태킹 기법

- 앙상블의 기본 알고리즘은 결정 트리

## Voting Classifier

- 하드보팅 : 다수결 원칙, 다수의 분류기가 결정한 예측값을 최종 보팅 결과값으로 선정
- 소프트 보팅 : 분류기들의 레이블 값 결정 확률을 모두 더해서 평균하고, 이들 중 가장 높은 레이블 값을 최종 보팅 결과값 으로 선정

In [27]:
import pandas as pd
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

cancer = load_breast_cancer()
cancer.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [28]:
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier()

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기

vo_clf = VotingClassifier(estimators=[('LR',lr_clf), ('KNN',knn_clf)], voting='soft')

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                   test_size = 0.2, random_state=156)

# votingclassifier 학습 평가

vo_clf.fit(X_train,y_train)
vo_clf_pred = vo_clf.predict(X_test)
print('vo_clf 정확도 : {:.4f}'.format(accuracy_score(y_test,vo_clf_pred)))


# 개별 모델의 학습/예측/평가
classifiers = [lr_clf, knn_clf]

for classifier in classifiers:
    classifier.fit(X_train,y_train)
    pred = classifier.predict(X_test)
    
    class_name = classifier.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test,pred)))

vo_clf 정확도 : 0.9474
LogisticRegression 정확도: 0.9386
KNeighborsClassifier 정확도: 0.9035


In [29]:
import pandas as pd

def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('dataset/human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 세트와 테스트 피처 데이터를 DataFrame으로 로딩, 칼럼명은 feature_name 적용
    # 우선 칼럼명 없이 데이터프레임으로 불러온 후 칼럼명을 추가함
    X_train = pd.read_csv('dataset/human_activity/train/X_train.txt', sep='\s+', header=None)
    X_train.columns = feature_name
    X_test = pd.read_csv('dataset/human_activity/test/X_test.txt', sep='\s+',header=None)
    X_test.columns = feature_name
    
    # 학습 레이블과 테스트 레이블 데이터를 DataFrame으로 로딩하고 칼럼명은 action으로 부여
    y_train = pd.read_csv('dataset/human_activity/train/y_train.txt', sep='\s+', header=None, names = ['action'])
    y_test = pd.read_csv('dataset/human_activity/test/y_test.txt', sep='\s+', header=None, names = ['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = get_human_dataset()

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
rf.fit(X_train,y_train)
predict = rf.predict(X_test)

accuracy_score(y_test,predict)

0.9219545300305395

## GBM ( Gradient Boosting Machine )

- 부스팅 알고리즘은 여러 개의 약한 학습기(weak learner)를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식

- 가중치 업데이트를 경사 하강법(Gradient Descent)를 이용한다

- 분류는 물론이고 회귀도 가능

- 파라미터 : n_estimators, max_depth, max_features
    - loss : 경사 하강법에서 사용할 비용함수 지정, 기본값 deviance 적용
    - learning_rate : GBM이 학습할 때 마다 적용할 '학습률', 오류값 보정 시 적용하는 계수로 0~1 사이의 값 지정, 기본값은 0.1
                      작게 설정하면 예측성능이 높아지나 수행시간이 오래 걸리고 큰 값을 적용하면 예측 성능이 쩔어질 가능성이
                      높으나 빠른 수행이 가능, n_estimator와 상호 보완적으로 조합해 사용
    - n_estimator : weak learner 의 개수
    - subsample : weak learner 가 학습에 사용하는 데이터의 샘플링 비율, 기본값은 1

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time

X_train, X_test, y_train, y_test = get_human_dataset()

start_time = time.time()

gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train,y_train)
gb_clf_preds = gb_clf.predict(X_test)

gb_clf_preds
print('예측정확도: ', accuracy_score(y_test,gb_clf_preds))
print('소요시간: ', time.time()-start_time)

예측정확도:  0.9389209365456397
소요시간:  638.819005727768


In [26]:
# 주의 ::  30분 ~ 1시간 소요
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100,200],
    'learning_rate':[0.05,0.1]
}

grid_cv = GridSearchCV(gb_clf,param_grid = params, cv=2, verbose=1)
grid_cv.fit(X_train,y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도:{:.4f}'.format(grid_cv.best_score_))

Fitting 2 folds for each of 4 candidates, totalling 8 fits
최적 하이퍼 파라미터:
 {'learning_rate': 0.05, 'n_estimators': 200}
최고 예측 정확도:0.9007


In [ ]:
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = gb_accuracy_score(y_test, gb_pred)

print('GBM 정확도:{:.4f}'.format(gb_accuracy))